In [18]:
#Dependencies
import http.client
import pandas as pd
import json
import numpy as np
import datetime as dt
import os
import pyodbc
import openpyxl
import sqlalchemy as db

In [19]:
#Set Up Connections
########## EpiInfo ##########
conn = http.client.HTTPSConnection("epi-info-data-service.services.cdc.gov")
conn.set_debuglevel(1)
headers = {
    'authtoken': "Xuv5NBkeYElE0ICxPdTTzw==",
    'accept': "*/*"
    }
########## PDL Database ##########
#Connect to PDL database (SQL SERVER through ODBC)
pdl_engine = db.create_engine("mssql+pyodbc://@Pertussis Lab Database", use_setinputsizes=False)

# #Initiate SQLAlchemy Metadata and table objects - not sure if needed
# metadata = db.MetaData()
# coryne = db.Table(
#     'coryne', 
#     metadata,
#     autoload_with=pdl_engine
# )
# input = db.Table(
#     'Input', 
#     metadata,
#     autoload_with=pdl_engine
# )

In [20]:
#Pull in CCRF data
conn.request("POST", "/api/SurveyData/GetSurveyData?surveyid=0c8b2402-03a4-4599-824c-2bd3a0fe6ab6", headers=headers)

res = conn.getresponse()
r = res.read()
data=json.loads(r.decode("utf-8"))
ccrf_df=pd.DataFrame(data)
pd.set_option('display.max_columns', 500)
#df[df['_Status']=='Complete']

send: b'POST /api/SurveyData/GetSurveyData?surveyid=0c8b2402-03a4-4599-824c-2bd3a0fe6ab6 HTTP/1.1\r\nHost: epi-info-data-service.services.cdc.gov\r\nAccept-Encoding: identity\r\nContent-Length: 0\r\nauthtoken: Xuv5NBkeYElE0ICxPdTTzw==\r\naccept: */*\r\n\r\n'
reply: 'HTTP/1.1 200 OK\r\n'
header: date: Thu, 25 Jan 2024 21:34:36 GMT
header: content-type: text/plain; charset=utf-8
header: server: Kestrel
header: content-length: 434388
header: set-cookie: 9765ae3492289acaddcb56a478ee7257=93f11089d79aed50b5521a777df68a53; path=/; HttpOnly; Secure; SameSite=None


In [5]:
#PDL Data Clean
#Select All PC IDs, add metadata
input_df = pd.read_sql_table('Input', pdl_engine)
coryne_df = pd.read_sql_table('coryne', pdl_engine)

In [6]:
#Clean Input
#input_df.columns
input_keep_cols=['ID','acc_num','age_lab','age_units','daterec','datecol','state_lab','spec_type','spec_type_oth']
input_clean = input_df[input_df['ID'].str.startswith('PC')][input_keep_cols].sort_values('ID',ascending=False)
#input_clean.head()

In [7]:
#Clean Coryne Results
#coryne_df.columns
coryne_keep_cols=['cdcid','toxigenic','cryne_ov_interp','coryne_pcr_interp','biotype','Coryne_biotype_oth']
coryne_clean = coryne_df[coryne_df['cdcid'].str.startswith('PC')][coryne_keep_cols].sort_values('cdcid',ascending=False)
#coryne_clean.head()

In [15]:
#Merge Lab Data
pdl_df= pd.merge(left=input_clean, right=coryne_clean, left_on='ID',right_on='cdcid',how='left')
pdl_df= pdl_df.drop(['cdcid'],axis=1).rename({'ID':'cdcid'},axis=1)
#pdl_df.head()

In [21]:
#Isolate complete CCRF responses, remove dummy data (GAEXAMPLE) and merge with PDL data
ccrf_df=ccrf_df[(ccrf_df['_Status']=='Complete') & (ccrf_df['RecordID'] != 'GAEXAMPLE')]



_Status
Complete    116
Name: count, dtype: int64

,ReportDate,datecolunk1,testtype_maldi1,testtype_cult1,testtype_unk1,testtype_oth1,specna2,datecolunk2,testtype_cult2,testtype_maldi2,testtype_unk2,testtype_oth2,specna3,datecolunk3,testtype_cult3,testtype_maldi3,testtype_unk3,testtype_oth3,GlobalRecordId,_DateUpdated,_Status,_ParentRecordId,RecordID,StateID,state,poc_email,Age,agetype,Sex,spectype1,datecol1,collectsite1,species1,Polymicrobial,CoInfections,RespNone,RespUnk,Fever,Voice,SoreThroat,Stridor,Weakness,Swallow,Fatigue,ShortBreath,NeckSwelling,Pseudomembrane,Wheeze,OthResp,CutNone,CutUnk,Serous,Sanguin,Serosang,fibulcer,plesion,purdrain,chronicwound,Abscess,ulcer,Swelling,Pustule,singlesion,redness,Vesicle,multlesion,OthCut,CompNone,CompUnk,Myocarditis,Bacteremia,Endocarditis,Airway,ekg,Neuropathy,CondNone,CondUnk,Alcohol,AtopDerm,Sinusitis,drugcurrent,Osteomyelitis,Cirrhosis,Herpes,CVD,RHD,HIV,Diabetes,Sepsis,OtherImmuno,Bullosa,HepC,OthCond,AST,ASTPattern,Erythro,Penicillin,Tetra,Amox,Azithro,AzithroDOI,AzithroDura,clarithro,Trimet,Cipro,Cephalexin,CephaDOI,CephaDura,Vancomycin,VancoDOI,VancoDura,UnkAbx,OtherAbx,vaccine,datelastvax,datelastvaxunk,datevax1,vaxtype1,datevax2,vaxtype2,datevax3,datevax4,vaxtype4,datevax5,Travel1,Housing,AnimalExposure,IVDU,CaseExposure,UnpasteurizedDairy,Dispo,DispoOther,vaxtype3,vaxtype5,sphlid1,CaseSummary,OthRespSpecify,OthCutSpecify,OthCondSpecify,CiproDOI,CiproDura,OtherDOI,OtherDura,OtherAbxSpecify,PenDOI,PenDura,vaccinenotes,collectsite2,datecol2,spectype2,sphlid2,species2,collectsite3,spectype3,datecol3,sphlid3,species3,ErythroDOI,ErythroDura,TetraDOI,TetraDura,AmoxDOI,AmoxDura,ClarithroDOI,ClarithroDura,TrimetDOI,TrimetDura,UnkDOI,UnkDura,datevax6,vaxtype6,datevax7,datevax8,vaxtype8,Placevisited1,Arrival1,Departure1,Placevisited2,Arrival2,Departure2,Placevisited3,Arrival3,Departure3,vaxtype7
2,2024-01-25,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,a5aaf73c-f169-4120-991a-3e395b1e8d4f,01/25/2024 15:06:18,Complete,,OK420473198,420473198,OK,Ashlyn Wayman,75.0,Years,Woman,2,2023-12-05,Lower Leg,C. diphtheriae,No,NaN,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,True,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,No,NaN,False,False,False,False,False,NaN,NaN,False,False,False,False,NaN,NaN,False,NaN,NaN,False,False,"Yes (Verbal report, records unavailable)",NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No - No Domestic/International Travel,Not experiencing homelessness,No exposure to animals,No drug use,No,Unknown,Discharged,NaN,NaN,NaN,23Y0074939,Case was seen for a fall that occurred in her ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-01-25,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,5d9c2994-d314-41a4-a316-af76a4467181,01/25/2024 12:10:34,Complete,,VA36955837,36955837,VA,emily.rich@vdh.virginia.gov,45.0,Years,Man,2,2024-01-11,left foot/ankle wound,C. diphtheriae,Yes,Moderate growth of Streptococcus pyogenes; Ver...,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,Yes,Susceptibility pending,False,False,False,False,False,NaN,NaN,False,False,True,False,NaN,NaN,True,2024-01-11,6,False,True,Unknown,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,Not experiencing homelessness,Dogs,Current IVDU,Unknown,Unknown,Discharged,NaN,NaN,NaN,NaN,Case form c